In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import zconfint
from statsmodels.stats.weightstats import *
from statsmodels.stats.descriptivestats import sign_test
from sklearn import cross_validation, linear_model, ensemble, metrics

C:\Users\gruyuri\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Выживаемость пациентов с лейкоцитарной лимфомой

In [2]:
data = pd.read_csv('Data_1.txt', sep = '\t', header = 0)
data.head(5)

,age
0,49
1,58
2,75
3,110
4,112


In [3]:
data.age_s = data.age - 200

In [4]:
stats.wilcoxon(data.age_s)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

# Влияние лесозаготовки на биоразнообразие лесов

In [5]:
data1 = pd.read_csv('Data_Tree_1.txt', sep = '\t', header = 0)
data2 = pd.read_csv('Data_Tree_2.txt', sep = '\t', header = 0)
data1.head()

,N
0,22
1,22
2,15
3,13
4,19


In [6]:
data2.shape

(9, 1)

In [7]:
stats.mannwhitneyu(data1.N, data2.N, alternative = 'greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

# Шаттл "Челленджер"

In [9]:
data_Space = pd.read_csv('challenger.txt', sep = '\t', header = 0)
data_Space.head(5)

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [14]:
ds_0 = data_Space[data_Space['Incident']  == 0].Temperature.values
ds_1 = data_Space[data_Space['Incident']  == 1].Temperature.values

In [11]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [12]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [15]:
np.random.seed(0)

ds0_scores = map(np.mean, get_bootstrap_samples(ds_0, 1000))
ds1_scores = map(np.mean, get_bootstrap_samples(ds_1, 1000))

In [17]:
delta_mean_scores = map(lambda x: x[1] - x[0], zip(ds1_scores, ds0_scores))
print "95% confidence interval for the difference between means",  stat_intervals(delta_mean_scores, 0.05)

95% confidence interval for the difference between means [ 1.42299107  7.93861607]


In [18]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [19]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [20]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [21]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [22]:
np.random.seed(0)
print "p-value: %f" % permutation_test(ds_1, ds_0, max_permutations = 10000)

p-value: 0.005700
